In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys 
from torch.nn import functional as F
import configparser
import pandas as pd 

import numpy as np
import torch
import matplotlib.pyplot as plt

import matplotlib
# import evaluate
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
from os.path import join as oj
sys.path.insert(0, "../src")
import my_data
import pickle as pkl
import predict
pd.set_option('display.float_format', lambda x: '%.3f' % x)
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
config = configparser.ConfigParser()
config.read('../config.ini');
data_path =config['DATASET']['data_path'] 
fig_path =config['PATHS']['figure_path'] 
import segmentation_models_pytorch as smp


In [ ]:
# need exp with no active learning
# need exp with prop increasingf
fnames = sorted([oj(config['PATHS']['model_path'] , fname) for fname in os.listdir(config['PATHS']['model_path'] ) if "pkl" in fname]) 
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects()


results.reset_index(inplace=True)

results[['foldername',  'cost_function', 'add_ratio',  'final_dice_score', 
         'experiment_name']].groupby([ 'experiment_name', 'cost_function','add_ratio', 
         'foldername' ]).mean()


In [15]:
from torch.utils.data import TensorDataset
import my_data
x, y, num_classes, class_dict = my_data.load_layer_data( oj(config["DATASET"]["data_path"], 'lno') )
data_min, data_max = np.min(x[:-1]), np.max(x[:-1])
x_test, y_test = x[-1:], y[-1:]
x_pool, y_pool = x[:-1], y[:-1]
x_pool_all, y_pool_all = my_data.make_dataset( x_pool, y_pool, img_size=128, offset=128, )
pool_set = TensorDataset( *[ torch.from_numpy(x_pool_all), torch.from_numpy(y_pool_all), ] )

In [24]:
net = smp.Unet(
    encoder_name="resnet34", encoder_weights=None,  
    in_channels=1,                  
    classes=3,
).to(device=device)

In [16]:
net = smp.UnetPlusPlus(
    encoder_name="resnet34", encoder_weights='imagenet',  
    in_channels=1,                  
    classes=3,
).to(device=device)

In [ ]:
results_now = results[results['experiment_name'] == 'ActiveLearning']
results_now = results_now[results_now.cost_function == 'emc']
results_now = results_now[results_now.add_ratio == 0.02]
net.load_state_dict(torch.load(oj(config['PATHS']['model_path'],  results_now.file_name.iloc[0] + '.pt')))


In [ ]:
a = predict.run(net, x_test, data_min, data_max, 512, 3, use_orig_values=True, return_whole = True )


In [ ]:
print("outside " + str(np.round((a.argmax(0) == 0).mean(), decimals =3)))


print("porosity " + str(np.round((a.argmax(0) == 1).sum() / (a.argmax(0) != 0).sum(), decimals =3)))


print("lno " + str(np.round((a.argmax(0) == 2).sum() / (a.argmax(0) != 0).sum(), decimals =3)))

# Test run time of both cost functions

In [10]:

from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import evaluate
loader_args = dict( batch_size=128, num_workers=1, pin_memory=True )
pool_loader = DataLoader(pool_set, shuffle=False, **loader_args)

In [11]:

from focal_loss.focal_loss import FocalLoss

criterion = FocalLoss(gamma=0.7, ignore_index=255)

In [ ]:
import time
start = time.time()
a = evaluate.emc(net, device, pool_loader, (data_min, data_max), n_choose=-1, num_classes = 3, criterion = criterion, )
print(time.time() - start)

In [ ]:
start = time.time()
a = evaluate.cut_off_cost(net, device, pool_loader, (data_min, data_max), n_choose=-1, num_classes = 3, criterion = criterion, )
print(time.time() - start)

In [ ]:
net.decoder
